| <img src ='https://bit.ly/39XPeCc' width=200 align='left'> |
|:--|
|<h1>    xtra_datacut_final_71_games.ipynb
</h1>
|<b>    1.0 데이터 전처리 테스트용 마지막 71게임 Data
</b>
| - 총 게임수 = 38,871 이므로 마지막 71 데이터로 전처리 테스트 후
| - 전체 게임으로 전처리를 진행한다.
| - 파일다루기 - https://parkdream.tistory.com/42
| - /content/drive/My Drive/Colab Notebooks/competition_data/starcraft_data/temp

In [1]:
"""
### 1.0 gdrive 마운트 / Import + 함수정의
- 코드를 사용해서 모듈 import 로 마운트 하거나
- 코랩 폴더에서 드라이브 마운트 클릭 하거나
"""
# from google.colab import drive
# drive.mount('/content/drive')

'\n### 1.0 gdrive 마운트 / Import + 함수정의\n- 코드를 사용해서 모듈 import 로 마운트 하거나\n- 코랩 폴더에서 드라이브 마운트 클릭 하거나\n'

# Multi-Processing
- 데이터 전처리에 메모리도 많이 먹고, 시간도 오래걸림
- 병렬처리하면 5배 이상 빨라진다니 한번 시도해 봄

In [2]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import multiprocessing as mp
from multiprocessing import Pool

num_cores = mp.cpu_count()               # cpu의 코어 수를 반환
print(f"THIS CPU CORES ARE .. = {num_cores} CORES!")        # CPU CORES = 16 CORES!

THIS CPU CORES ARE .. = 4 CORES!


## 사용함수의 정의

In [4]:
""" 
# 기본적인 module import 위치로 작업폴더 변경 getcwd() --> chidr()
# HOME 을 지정하고 작업폴더를 HOME 으로 변경.
"""
import os

HOME= 'dacon_competition_2020'
dir_base = "".join(os.getcwd().partition(HOME)[:2]) + "\\"
os.chdir(dir_base)     

In [5]:
from _assets.config_dirs import *
from _assets.modules import *


# 화일분석에 필요한 공동폴더를 등록합니다.
- asset = dir_base + '_assets/'
- statics dir_base + '_statics/'

#  echo = True : SYS.PATH INSERT 상황 보여줌


# OS 화일 및 DF 정보조회를 위한 탐색 모듈



In [6]:
from _assets.module_data_preps import ( __doc__,
                                tqdm,  
                                species_converter, 
                                data_preparation,
                            )


# GAME_ID 유니크 데이터 전처리를 위한 모듈 - baseline offer



In [8]:
show_ls(dir_base + remake )

False
DIR_TARGET=C:\Users\nitt0\Documents\Github\dacon_competition_2020\_assets
C:/Users/nitt0/Documents/Github/dacon_competition_2020/c03_starcraft_prediction/data_remake/
----------------------------------------
  01. .ipynb_checkpoints
  02. accumujated_winning_rate_per_time.csv
  03. accumulated_winning_rate_per_time.csv
  04. df_new_rate04_correct.csv
  05. df_win_rate_every_10sec.csv
  06. df_win_rate_every_20sec.csv
  07. df_win_rate_every_40sec.csv
  08. df_xtest_remake.csv
  09. df_xtest_remake_final.csv
  10. df_xtrain_remake.csv
  11. df_xtrain_remake_final.csv
  12. df_ytrain_remake.csv
  13. xtra_remake_xtest.csv
  14. xtra_remake_xtest_r01.csv
  15. xtra_remake_xtrain.csv
  16. xtra_remake_xtrain_r01.csv


## DATA 분할 of final 71s

In [16]:
%%time
"""
# *** DATA SHAPE = [ 67,091,776 x 7 ] ... 가볍다!
# CPU times: user 42.5 s, sys: 3.63 s, total: 46.1 s
# Wall time: 43.2 s
"""

df_train = pd.read_csv(dir_base + raw + 'train.csv')
show_infoDF_from(df_train)

*** DATA SHAPE = [ 67,091,776 x 7 ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         int64  
 1   winner          int64  
 2   time            float64
 3   player          int64  
 4   species         object 
 5   event           object 
 6   event_contents  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 3.5+ GB
None

            game_id        winner          time        player
count  6.709178e+07  6.709178e+07  6.709178e+07  6.709178e+07
mean   1.944981e+04  5.012076e-01  4.277912e+00  4.992699e-01
std    1.122980e+04  4.999985e-01  2.709741e+00  4.999995e-01
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
25%    9.718000e+03  0.000000e+00  2.120000e+00  0.000000e+00
50%    1.948500e+04  1.000000e+00  4.150000e+00  0.000000e+00
75%    2.919700e+04  1.000000e+00  6.360000e+00  1.000000e+00
max    3.887100e+04  1.0

In [ ]:
show_ls(dir_base + remake)

In [18]:
"""
# *** DATA SHAPE = [ 114,504 x 7 ] ... 1/590 배 가볍다!
"""
df_train_cut71 = df_train[df_train['game_id'] > 38800]
show_infoDF_from(df_train_cut71)  

del(df_train)       #  더이상 사용하지 않는 df는 꼭 삭제할것 = 메모리 낭비 방지

NameError: name 'df_train' is not defined

## 이벤트 Detail 탐색을 위해 각 이벤트 counts dict를 만든다

In [24]:
%%time
"""
# 이벤트 분류를 위한 counts_dict 를 만든다.
# CPU times: user 75.4 ms, sys: 2.81 ms, total: 78.2 ms
# Wall time: 77.6 ms
"""
counts = get_counts_dict_fromDF(df=df_train_cut71)

Wall time: 244 ms


In [25]:
counts['Camera']

at (38.5, 120.7578125)          218
at (161.5, 19.7578125)          174
at (129.5, 24.7578125)          146
at (125.5, 28.7578125)          126
at (133.5, 32.7578125)          116
                               ... 
at (33.42578125, 78.9296875)      1
at (66.703125, 107.37109375)      1
at (37.7109375, 28.3203125)       1
at (140.578125, 25.44140625)      1
at (66.34765625, 92.0546875)      1
Name: event_contents, Length: 47418, dtype: int64

In [26]:
counts['Right Click']

Target: None [023C0001]; Location: (19.0, 138.5, 49136)                             80
Target: DestructibleSignsConstruction [001C0001]; Location: (32.0, 122.5, 49088)    71
Target: DestructibleSignsConstruction [02440001]; Location: (21.0, 142.5, 49136)    69
Target: DestructibleSignsConstruction [02B40001]; Location: (132.0, 30.5, 49136)    59
Target: None [00040001]; Location: (38.0, 129.5, 49088)                             50
                                                                                    ..
Location: (148.73291015625, 161.841064453125, 49120)                                 1
Location: (42.207275390625, 96.353759765625, 40941)                                  1
Target: Adept [040C0002]; Location: (133.923828125, 158.1640625, 49104)              1
Location: (85.587158203125, 76.0751953125, 33358)                                    1
Location: (142.2626953125, 158.98779296875, 49105)                                   1
Name: event_contents, Length: 16333, dtype:

## rightClick(DF)를 세미콜론 기준, 스플릿하는 멀티프로세싱
- 새로 컬럼을 만든다.

In [27]:
def get_basic_df(dict, key_name='Right Click'):
    """counts 딕트에서 기본 key df 를 만들고 컬럼 제정렬 한다."""
    df = pd.DataFrame(dict[key_name])
    df['details'] = df.index.copy()
    df.index = np.arange(len(df['details']))
    df = df[['details', 'event_contents']]              # re-order column position 

    # df.columns = ['details', 'counts']                # rename ... not shows result
    df.rename(columns={'event_contents' : 'counts'})    # rename ... shows result [추천]
    return df

In [28]:
def parallelize_dataframe(df, func):
    """DF 를 코어수만큼 쪼개서, 매핑하고 다시 연결시킴
    Pandas DataFrame을 병렬처리 하는 방법 = https://bit.ly/2JKTw5v
    """
    num_cores = mp.cpu_count()               # cpu의 코어 수를 반환
    print(f"CPU CORES = {num_cores} CORES!")
    
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)

    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [29]:
# 전처리 필요합수를 재지정 한다.
def func_helper(df):
    """HELPER() for multi-process = 처리함수를 별도수정 필요"""
    df['num_split'] = df['details'].apply(lambda x: len(x.split(';')))
    df['splits_01'] = df['details'].apply(lambda x: x.split(';')) 
    df['splits_02'] = df['splits_01'].apply(lambda x: x[0].split(':')[1]) 
    return df

In [30]:
df_rightClick = get_basic_df(dict=counts, key_name='Right Click')
df_rightClick.head()

,details,event_contents
0,"Target: None [023C0001]; Location: (19.0, 138....",80
1,Target: DestructibleSignsConstruction [001C000...,71
2,Target: DestructibleSignsConstruction [0244000...,69
3,Target: DestructibleSignsConstruction [02B4000...,59
4,"Target: None [00040001]; Location: (38.0, 129....",50


In [ ]:
%%time
df_multi = parallelize_dataframe(df_rightClick, func_helper)

CPU CORES = 4 CORES!


In [ ]:
# df_multi.sort_values(by='num_split', ascending=False).head(10)
cols = ['event_contents', 'num_split', 'splits_01', 'splits_02', ]
df_multi[cols].sort_values(by='event_contents', ascending=False).head(50)

In [ ]:
df_Ability = get_basic_df(dict=counts, key_name='Ability')
df_Ability.head(10)

In [ ]:
# 전처리 필요합수를 재지정 한다.
def func_helper(df):
    """HELPER() for multi-process = 처리함수를 별도수정 필요"""
    df['num_split'] = df['details'].apply(lambda x: len(x.split('-')))
    df['splits_01'] = df['details'].apply(lambda x: x.split('-')) 
    df['splits_02'] = df['splits_01'].apply(lambda x: x[0])
    df['splits_03'] = df['splits_01'].apply(lambda x: x[-1])  
    return df

In [ ]:
%%time
df_multi_Ability = parallelize_dataframe(df_Ability, func_helper)

In [ ]:
# df_multi.sort_values(by='num_split', ascending=False).head(10)
cols = ['event_contents', 'num_split', 'splits_01', 'splits_02', 'splits_03',]
df_multi_Ability[cols].sort_values(by='event_contents', ascending=False).head(50)

In [ ]:
# 셀렉션의 최빈값이 NULL 이다 = 무슨의미 일까? --- 모르면 feature로 넣어보고 평가!
counts['Selection'][:50]

## 공격,생산을 코드별로 분류해서 카운트 한다
- 질문 : 공격코드가 많을까? 생산코드가 많을까? 추측 = 진화코드가 많을까?
- 예) 5A0 = Attack; Target = Object; Location=좌표 ---> 공격 코드
- 단일코드) 1360 = TrainSCV / 1820=MorphDrone / 15E0 = TrainProbe = 생산유닛 뽑기를 하나로 묶음!


In [ ]:
counts['Ability']

In [ ]:
counts['Ability'][:60]

In [ ]:
counts['Ability'][-60:-1]

# Null 타입 이벤트 (공격이벤트)
1. Ability
> - 어택 횟수가 공격에 미치는 영향
> - 훈련 횟수가 공격에 미치는 영향

1. 공격 이벤트 = Null 값 
> - 어느 이벤트가 더 영향을 미칠까? 모르겠으면 다 합한다.
> - 부대 지정횟수
> - 부대 추가횟수
> - 부대 불러오기 횟수
> - 부대 행동 횟수

1. Selection = Null 값 빈도가 제일 높음?
> - Null 값의 의미가 뭘까? 모르면 플레이어별로 카운트
> - Null 값이 + or - 영향인지 모름


| 컬럼명  | 값    | type             | 설명           | 대표값 |   비고 |
|:-------:|:------|------------------|----------------|:-------|:------:|
| event   |Ability           |Object |생산, 공격 등 선수의 주요 행동 | (code) | 어택, 훈련횟수 추출!
|         |Camera            |Object |시점 선택               | at(coords)  | 첫시점=시작포인트!
|         |SetControlGroup   |Object |부대 지정       |       NULL        
|         |AddToControlGroup |Object |부대에 추가     |       NULL        
|         |GetControlGroup   |Object |부대 불러오기   |       NULL        
|         |ControlGroup      |Object |부대 행동       |       NULL        
|         |Right             |Object |Click : 마우스 우클릭   | Target:(object); Location: at(coords)
|         |Selection         |Object |객체 선택       |NULL 값 추출 | 종류 너무 많음! 

In [ ]:
counts['SetControlGroup']

In [ ]:
counts['GetControlGroup']

In [ ]:
counts['AddToControlGroup']

In [ ]:
counts['ControlGroup']

## BaseLine Data Preps = df_preps

In [ ]:
%%time
"""
# 베이스라인에 제시된 game_id 할당 전처리 수행
# CPU times: user 573 ms, sys: 12 ms, total: 585 ms 
# Wall time: 751 ms 
"""
x_train, y_train = data_preparation(df_train_cut71, answer=True)

In [ ]:
print(data_preparation.__doc__)
print(type(x_train))   # <class 'pandas.core.frame.DataFrame'>
print(type(y_train))   # <class 'numpy.ndarray'>

In [ ]:
y_train.shape       # (38872,)
pd.Series(y_train).value_counts()

In [ ]:
show_infoDF_from(x_train)
x_train.head(3)

In [ ]:
y_train     # 71개 승리데이터

In [ ]:
serieses = [
        x_train.P0_AddToControlGroup,
        x_train.P1_AddToControlGroup,
        x_train.P0_ControlGroup,
        x_train.P1_ControlGroup,
    ]
plot_it(serieses, 1,1, figsize=(8,3), overlap=True)

In [ ]:
x_train.shape   # (71, 26)  ... DataFrame  문제지 26열
y_train.shape   # (71,)     ... np.array   답안지 1열

In [ ]:
x_train['P0_total_null'] =  x_train['P0_AddToControlGroup'] + \
                            x_train['P0_GetControlGroup'] + \
                            x_train['P0_SetControlGroup'] + \
                            x_train['P0_ControlGroup']

x_train['P1_total_null'] =  x_train['P1_AddToControlGroup'] + \
                            x_train['P1_GetControlGroup'] + \
                            x_train['P1_SetControlGroup'] + \
                            x_train['P1_ControlGroup']

x_train['delta_total_null'] =   x_train['P0_total_null'] - \
                                x_train['P1_total_null']


In [ ]:
x_train.shape      # (71, 29)
x_train.head(4)